# **Object Detection**

<a target="_blank" href="https://github.com/amaciag/CV-Experiments/blob/main/Shampoo_Object_Detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"/>View source on GitHub</a>

Object Detection is the supervised learning process of locating and identifying objects in an image or video. This tutorial shows how to use some Google tensorflow pretrained model to train on a shampoo image dataset. Transfer learning and data augmentation techniques are incorporated.

## **Install and import appropriate modules and their dependencies**

In [ ]:
import tensorflow as tf
from google.colab import drive
from IPython.display import Javascript

def resize_output_cell():
  display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

get_ipython().events.register('pre_run_cell', resize_output_cell)
drive.mount('/content/drive')

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
%%writefile models/research/object_detection/packages/tf2/setup.py
"""Setup script for object_detection with TF2.0."""
import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    # Required for apache-beam with PY3
    'avro-python3',
    'apache-beam',
    'pillow',
    'lxml',
    'matplotlib',
    'Cython',
    'contextlib2',
    'tf-slim',
    'six',
    'pycocotools',
    'lvis',
    'scipy',
    'pandas',
    'tf-models-official==2.7.0',
    'tensorflow_io',
    'keras'
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)



In [ ]:
%%bash

sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
pip install opencv-python-headless==4.1.2.30

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1684828 sha256=a201a22731ab98f4be5ed4537899ac99c41d2bd2a99cff6d53a05d24ab469354
  Stored in directory: /tmp/pip-ephem-wheel-cache-kbxgobhu/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1




  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
%%writefile generate_tf_record.py
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import argparse
import os
import io
import json
import pandas as pd
import requests
import tensorflow as tf
import sys
# sys.path.append("./models/research")

from PIL import Image
from object_detection.utils import dataset_util, label_map_util

def create_tf_example(data):
    
    # print(data['Labeled Data'])
    response = requests.get(data['Labeled Data'])
    encoded_jpg = response.content
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for obj in data['Label']['objects']:
        bbox = obj['bbox']
        xmin = bbox['left']
        ymin = bbox['top']
        xmax = xmin + bbox['width']
        ymax = ymin + bbox['height']
        xmins.append(xmin / width)
        xmaxs.append(xmax / width)
        ymins.append(ymin / height)
        ymaxs.append(ymax / height)
        classes_text.append(obj['title'].encode('utf8')) 
        classes.append(label_map_dict[obj['title']])
    
    filename = data['External ID'].encode('utf8')
    image_format = b'jpg'
    tf_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/width': dataset_util.int64_feature(width),
            'image/filename': dataset_util.bytes_feature(filename),
            'image/source_id': dataset_util.bytes_feature(filename),
            'image/encoded': dataset_util.bytes_feature(encoded_jpg),
            'image/format': dataset_util.bytes_feature(image_format),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
    return tf_example

def main(args):
    
    with tf.io.gfile.GFile(args.json_input, 'r') as file:
        dataset = json.load(file)
    
    writer = tf.io.TFRecordWriter(args.output_path)
    for data in dataset:
        tf_example = create_tf_example(data)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print(f'Successfully created TF-records {args.output_path}')


if __name__ == '__main__':
    argparser = argparse.ArgumentParser()
    argparser.add_argument('--json-input')
    argparser.add_argument('--output-path')
    argparser.add_argument('--label-map')
    args = argparser.parse_args()
    
    label_map_dict = label_map_util.get_label_map_dict(args.label_map)
    main(args)

In [ ]:
%%bash

if [ ! -d workspace ]; then
  mkdir -p workspace/{pretrained_models,trained_models,data}
fi

wget -O workspace/data/eval.json https://raw.githubusercontent.com/amaciag/CV-Experiments/main/data/eval.json
wget -O workspace/data/train.json https://raw.githubusercontent.com/amaciag/CV-Experiments/main/data/train.json

In [ ]:
items = [{
  'id': '1',
  'name': '"Loreal"'
},
{
  'id': '2',
  'name': '"Pantene"'
},
{
  'id': '3',
  'name': '"Head_&_Shoulders"'
}]

expression = ''
for item in items:
  expression += 'item {\n'
  for key,value in item.items():
    expression += 2*' ' + key + ': ' + value + '\n'
  expression += '}\n'

with open('workspace/data/shampoo_label_map.pbtxt', 'w') as f:
  f.write(expression)
print(expression)

In [ ]:
!python generate_tf_record.py --json-input workspace/data/train.json --output-path workspace/data/train.record --label-map workspace/data/shampoo_label_map.pbtxt
!python generate_tf_record.py --json-input workspace/data/eval.json --output-path workspace/data/eval.record --label-map workspace/data/shampoo_label_map.pbtxt

In [ ]:
MODELS = {
    "EfficientDet D0 512x512": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz",
    "EfficientDet D2 768x768": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz"
}

selected_model = "EfficientDet D0 512x512"
file = MODELS[selected_model].split('/')[-1]

In [ ]:
!wget {MODELS[selected_model]} --directory-prefix workspace/pretrained_models
!tar -xf workspace/pretrained_models/{file} -C workspace/pretrained_models

--2022-01-27 15:20:52--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.5.128, 2a00:1450:400c:c1b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.5.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘workspace/pretrained_models/efficientdet_d0_coco17_tpu-32.tar.gz.1’

efficientdet_d0_coc 100%[===================>]  29.31M   168MB/s    in 0.2s    

2022-01-27 15:20:52 (168 MB/s) - ‘workspace/pretrained_models/efficientdet_d0_coco17_tpu-32.tar.gz.1’ saved [30736482/30736482]



In [ ]:
from object_detection.utils import config_util

model_dir = file.replace('.tar.gz', '')
pipeline_config = config_util.get_configs_from_pipeline_file(f'workspace/pretrained_models/{model_dir}/pipeline.config')

pipeline_config['model'].ssd.num_classes=3
pipeline_config['train_config'].batch_size=4
pipeline_config['train_config'].fine_tune_checkpoint_type='detection'
pipeline_config['train_config'].fine_tune_checkpoint=f'workspace/pretrained_models/{model_dir}/checkpoint/ckpt-0'
pipeline_config['eval_input_config'].label_map_path='workspace/data/shampoo_label_map.pbtxt'
pipeline_config['eval_input_config'].tf_record_input_reader.input_path[0]='workspace/data/eval.record'
pipeline_config['train_input_config'].label_map_path='workspace/data/shampoo_label_map.pbtxt'
pipeline_config['train_input_config'].tf_record_input_reader.input_path[0]='workspace/data/train.record'

pipeline_config = config_util.create_pipeline_proto_from_configs(pipeline_config)
config_util.save_pipeline_config(pipeline_config, 'workspace/trained_models')

INFO:tensorflow:Writing pipeline config file to workspace/trained_models/pipeline.config


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=drive/MyDrive/CV-Experiments-main/efficientdet_d0/v2

In [ ]:
!python models/research/object_detection/model_main_tf2.py --pipeline_config_path=workspace/trained_models/pipeline.config \
  --model_dir=drive/MyDrive/CV-Experiments-main/efficientdet_d0/v3 --checkpoint_every_n=10 --num_train_steps=100 --alsologtostderr

2022-01-27 17:03:02.271334: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0127 17:03:02.275178 140258431371136 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0127 17:03:02.280510 140258431371136 config_util.py:552] Maybe overwriting train_steps: 2000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0127 17:03:02.280679 140258431371136 config_util.py:552] Maybe overwriting use_bfloat16: False
I0127 17:03:02.292429 140258431371136 ssd_efficientnet_bifpn_feature_extractor.py:146] EfficientDet EfficientNet backbone version: efficientnet-b0
I0127 17:03:02.292589 140258431371136 ssd_efficientnet_bifpn_feature_extractor.py:147] 

In [ ]:
# LOG_DIR = 'workspace/efficientdet_d0/v1'
# get_ipython().system_raw(
#     'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#     .format(LOG_DIR)
# )

In [ ]:
# get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
# !curl -s http://localhost:4040/api/tunnels | python -c \
#   "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://44b8-35-205-166-217.ngrok.io


In [ ]:
%%bash
cp models/research/object_detection/exporter_main_v2.py .
python exporter_main_v2.py --input_type image_tensor \
  --pipeline_config_path workspace/trained_models/pipeline.config  \
  --trained_checkpoint_dir drive/MyDrive/CV-Experiments-main/efficientdet_d0/v2 \
  --output_directory sample_model

In [ ]:
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = './sample_model/saved_model'

detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

In [ ]:
import json
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams.update({'figure.max_open_warning': 0})

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

with tf.io.gfile.GFile('workspace/data/eval.json', 'r') as file:
        eval_data = json.load(file)

category_index = label_map_util.create_category_index_from_labelmap('./workspace/data/shampoo_label_map.pbtxt', use_display_name=True)

for data in eval_data:
    
    url = data['Labeled Data']
    response = requests.get(url)
    encoded = io.BytesIO(response.content)
    image_np = load_image_into_numpy_array(encoded)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    
    shampoo = category_index[detections['detection_classes'][np.argmax(np.max(detections['detection_multiclass_scores'], axis=1))]]['name']
    plt.figure(figsize=(20,20))
    plt.title(shampoo)
    plt.imshow(image_np_with_detections)
    # plt.cla()
    # print('Done')
plt.show()